Gerekli kütüphaneler yüklendi.

In [13]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains   
import time
from warnings import warn

Web scraping işlemini yapacağımız sitenin adresine ihtiyacımız var.

In [14]:
url = "https://www.linkedin.com/jobs/search/?geoId=102105699&keywords=Yaz%C4%B1l%C4%B1m&location=T%C3%BCrkiye"

Selenium web driver ile hangi tarayıcıda scraping işlemini yapacaksam ona uygun driver'ı kullanıyorum. Burada chromedriver'ın 
bulunduğu doğru dosya yolunu seçmek ve önemli. chromedriver'ı çalıştırmak amacında olduğum için dosya yolunun derlenebilir olduğuna dikkat edin ve .exe dosya uzantısını unutmayın. 'r' parametresi ile dosya yolundaki string'i dosya yolunu belirtirken \ ifadesinden kurtarıyorum . \n \t vs. python'da enter ve tab tuslarına karşılık geldiği için bu kullanımdan kaçıyorum.

In [15]:
driver = webdriver.Chrome(r"D:\Users\baran.turkmen\Desktop\chromedriver.exe")
driver.get(url)

Web scraping yapacağımız web sitesine eriştiğimizde verileri çekebilmek için giriş yapmamız lazım. Login alanlarının ve butonlarının yüklendiğinden emin olmak için time.sleep ile bir süre bekliyorum. Her zaman stabil şekilde otomatik olarak çalışmıyor. Bu durum internet bağlantısının kalitesinin düşük olmasından kaynaklanıyor olabilir. Bundan dolayı hata yakalama metodlarını kullanarak stabil çalışmadığı durumlarda tekrar kullanıcıyı giriş ve şifre yollama menüsüne yönlendiriyorum .Eğer yine olmazsa manuel olarak kullanıcı adı ve şifrenizi girip devam edebilirsiniz.

In [16]:
try:
    time.sleep(3)
    driver.find_element_by_xpath("/html/body/header/nav/div/a[2]").click()
    time.sleep(5)
    mail = input("Lütfen kayıtlı mail adresinizi giriniz : ")
    sifre = input("Lütfen şifrenizi giriniz : ")
    driver.find_element_by_id("username").send_keys(mail) #linkedin mail girilecek
    driver.find_element_by_id("password").send_keys(sifre)  #linkedin sifre girilecek
    driver.find_element_by_xpath("/html/body/div/main/div[2]/div[1]/form/div[3]/button").click()
except:
    time.sleep(3)
    driver.find_element_by_xpath("/html/body/header/nav/div/a[2]").click()
    time.sleep(5)
    driver.find_element_by_id("username").send_keys(mail) #linkedin mail girilecek
    driver.find_element_by_id("password").send_keys(sifre)  #linkedin sifre girilecek
    driver.find_element_by_xpath("/html/body/div/main/div[2]/div[1]/form/div[3]/button").click()

Lütfen kayıtlı mail adresinizi giriniz : baranemre1997@gmail.com
Lütfen şifrenizi giriniz : time.sleep(2644)


Sayfa açılınca otomatik olarak gelen mesajlaşma kısmını kapatıyorum. Veri çekerken ulaşmam gereken web elementleri var. Ortalığı karıştırmasını istemiyorum bu kısımın.

In [17]:
try:
    time.sleep(5)
    driver.find_element_by_xpath("(//*[@class='msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view'])[2]").click()
except:
    time.sleep(5)
    driver.find_element_by_xpath("(//*[@class='msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view'])[2]").click()

Çekilecek bilgileri depolamak için listeleri oluşturdum. Ayrıca oluşturduğum algoritma için bir tane de kontrol parametresi oluşturdum. Ana başlık listesi ilandaki ana başlıktan çektiğim bilgileri içerecek. Ana başlık kendi içinde her bir ilan için konum ve şirket ismini içeriyor. Bu liste üzerinden daha sonra ana başlık ve konumu ayıracağım. Yine aynı şekilde kıdem düzeyi, iş birimi, sektör ve istihdam bilgilerini tek bir listede toplama yoluna gittim . Bunun sebebi her bir ilan için bu bilgilerin dördünün de bulunmaması. Bulananları her bir ilan için alıyorum. Bulunmayalara None değer atıyorum. Tamamen oluşturulan algoritmaya göre alınmış bir karar. Bu bilgiler ayrı ayrı da elde edilebilir.

In [18]:
calisan_sayisi = [] 
basvuru_sayisi = []
ana_baslik = []
pozisyon = []
açıklama = []
kıdemd_isb_sek_ist = []
kontrol = 0

Aşağıda linkedin sayfa yapısına uygun bir şekilde bir algoritma tasarladım. Bu algoritmayı tasarlarken bol bol python'ın hata yakalama özelliğinden faydalandım .

for j in range(1,40) ile 1'den 40'a kadar gitmemim nedeni Yazılım anahtar kelimesi ile Linkedinde ilanların 39 sayfada toplanması.

Her yeni sayfaya geçişte time.sleep(5) ile 5 saniye bekliyorum.

for i in range(1,26): linkedinde her sayfada 25 ilan var. Fakat aslında bu durum öyle değil bazı sayfalarda ilan sayısı 23 bazı sayfalarda 10 vs. değişken bir durum vardı. Gözlemlerime gördüğüm maksimum 25 tane ilan oluşuydu. Yine sayım hatası yapmış olabileceğimden fazladan 2 ilan daha ekledim. İlanların fazla gelmesi durumunda hata yakalama sistemi devreye giriyor ve döngünün içinde var olmayan eksik ilanlar için herhangi bir şey elde etmeden boşa dönüp yeni sayfaya geçiyorum.

element = driver.find_element_by_xpath("//*[@class='artdeco-pagination__pages artdeco-pagination__pages--number']") ile sayfanın en aşağısında ki sayfa numaralarının bulunduğu web elementine gidiyorum. Sayfanın en aşağısına gitme durumu sadece her sayfaya yeni geldiğimiz zaman bir kere gerçekleştiriliyor. Bunu yapmamızın sebebi şu , eğer safyanın en aşağısına gitmezsem sayfada ki tüm ilanlar yüklenmiyor. O sayfa 20 tane ilan varsa eğer yaklaşık olarak 15 tane ilanı hiç yokmuş gibi kaçırıyoruz. Bu durumda aşağıda kalan ilanlara ulaşmak adına sayfanın aşağısında ki bi web elementine gidiyorum. Bu şekilde aşağılarda kalan kaybetme olasılığımın olduğu ilanlar yüklenmiş oluyor , görünürlükleri ortaya çıkıyor. Bunu scroll ile de yapabilirdik fakat scroll ile uğraşmaktansa aşağıda bir web elementine gidip tüm ilanların sayfada yüklenmesini sağlamak daha pratik.

element = driver.find_element_by_xpath("(//*[@class='disabled ember-view job-card-container__link job-card-list__title'])["+str(i)+"]") ile tıklamam gereken ilanları elde ediyorum. Sayfanın en aşağısından yukarıya çıkma işlemini ilk ilana ulaştığımda otomatik bir şekilde yapmış oluyorum. 

actions = ActionChains(driver) her bir ilanın üzerinde hareket edebilmem için bu koda ihtiyacım var.
actions.move_to_element(element).perform() ilanların üzerinde bu kod sayesinde hareket edebiliyorum.
element.click() ile ilanlara tıklıyorum. İlanlardan çekmek istediğim bilgileri elde edebilmem için o ilana tıklamam gerekli.

driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") sayfada var olan tüm ilanların sonuna geldiysem hata elde edeceğim , sayfa sonuna hareket etmemi sağlıyor bu kod hata elde ettiğim zaman. Amacımız linkedin'de her sayfada ilan sayısı tam olarak net olmadığından ve biz her sayfada 25 tane ilan varmış gibi yaptığımızdan 25'ten az olan ilanlar için aynı ilanları tekrar tekrar elde etmek. Daha sonra duplicate olan ilanları çıkaracağız. Bu arada bunun yapılmasının sebebi kaç adet ilan var her sayfada bilmiyorum ve oluşacak hata yüzünden döngüden çıkmak , sonraki sayfalara geçememek gibi bir durumla karşılaşmak istemiyorum.

İlanların üzerinde hareket edip tıklama yapabildiğime göre açılan her bir ilan için çekmek istediğim bilgileri çekiyorum. Bunu yaparken xpath'den fazlası ile faydalandım. Çekmek istediğim bilgileri elde edebilmem için web sitesi üzerinde var olan web elementine ulaşmam gerekli bundan dolayı driver.find_element_by_xpath komutunu kullandım. Eğer çekmek istediğim bilgi o ilanda yoksa None değer atıyorum o bilgiye. 

print(ksii.text) ile kıdem düzeyi ,iş birimi ,sektör ve istihdam türünden hangilerini elde ettiğimi ilan başına kontrol ediyorum. Projemiz için bu bilgiler fazlası ile önemliydi. 

kssi kullanmamım nedeni şu linkedinde kıdem düzeyi , çalışma tipi gibi bilgileri tutan xpathlerin bazen aynı adrese aynı xpath'e sahip olması. Bu durum kssi meselesi biraz veri kaybetmemize sebeb oluyor ama en azından verilerin doğruluğundan eminiz. 

if(j>8 and j<sayfa_no-7): ve elif(j>=sayfa_no-7): değerleri önemli. Sonraki sayfaya geçmek istediğim sonraki sayfaya geçmek için tıklamam gereken web elementinin adresi ilk 8 sayfa , 8-sayfa_no-7 sayfaları arasında ve sayfno-7'den en son sayfaya kadar farklı. Bulunduğumuz sayfaya göre 3 farklı adrese sahip. Bundan dolayı böyle bir yol izledim.

print("{}. sayfa".format(j)) ve print("-----------------------------------------------------------") ile otomasyon aşamasında kaçıncı sayfada olduğumu görebiliyorum.

Son durumda sayfalar arası geçişi sağlarken elif(j>=sayfa_no-7) bölümünde kalan son birkaç sayfada sayfalar arası geçişi sağlamak adına str(j-(sayfa_no-10)) parametresini xpath'in içerisinde göreceksiniz. Ve yine if(j>8 and j<sayfa_no-7) altında xpath'in içinde sadece 6 değeri ile sayfalar arası geçiş sağlanıyor.Bu durum Linkedin'in sayfa yapısından kaynaklanıyor.

//*[@class='t-16 t-bold t-black--light ph6']

t-16 t-black--light t-bold mb1

//*[@class='artdeco-pagination__pages artdeco-pagination__pages--number']

In [19]:
sayfa_no = int(driver.find_element_by_xpath("(//*[@class='artdeco-pagination__indicator artdeco-pagination__indicator--number ember-view'])[9]").text)
print("Sayfa sayısı : ",sayfa_no)

Sayfa sayısı :  35


In [20]:
for j in range(1,sayfa_no+1):
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath("//*[@class='artdeco-pagination__pages artdeco-pagination__pages--number']")
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
    except:
        try:
            driver.refresh()
            time.sleep(5)
            element = driver.find_element_by_xpath("//*[@class='artdeco-pagination__pages artdeco-pagination__pages--number']")
            actions = ActionChains(driver)
            actions.move_to_element(element).perform()
        except:
            pass
    for i in range(1,26):
        try:
            time.sleep(2)
            element = driver.find_element_by_xpath("(//*[@class='disabled ember-view job-card-container__link job-card-list__title'])["+str(i)+"]")
            actions = ActionChains(driver)
            actions.move_to_element(element).perform()
            time.sleep(2)
            element.click()
        except:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        try:
            #Poziyon
            pozisyon.append(element.text)
        except:
            pozisyon.append(None)
        try:
            #Basvuru sayisi
            bas_say = driver.find_element_by_xpath("(//*[@class='jobs-details-job-summary__text--ellipsis'])[1]").text
            basvuru_sayisi.append(bas_say)
        except:
            basvuru_sayisi.append(None)
        try:
            #Ana baslik -> Sirket adi , konum 
            ana=driver.find_element_by_xpath("(//*[@class='jobs-details-top-card__company-info t-14 t-black--light t-normal mt1'])").text
            ana_baslik.append(ana)
        except:
            ana_baslik.append(None)
        try:
            #Calisan sayisi
            cal_say = driver.find_element_by_xpath("(//*[@class='jobs-details-job-summary__text--ellipsis'])[3]").text
            calisan_sayisi.append(cal_say)
        except:
            calisan_sayisi.append(None)
        try:
            #kıdem düzeyi , sektör ,istihdam türü ,iş birimi
            time.sleep(2)
            ksii = driver.find_element_by_xpath("//*[@id='job-details']/div[2]")
            actions = ActionChains(driver)
            actions.move_to_element(ksii).perform()
            time.sleep(2)
            kıdemd_isb_sek_ist.append(ksii.text)
        except:
            try:
                time.sleep(2)
                ksii = driver.find_element_by_xpath("//*[@id='job-details']/div[1]")
                actions = ActionChains(driver)
                actions.move_to_element(ksii).perform()
                time.sleep(2)
                kıdemd_isb_sek_ist.append(ksii.text)
            except:
                kıdemd_isb_sek_ist.append(None)
                kontrol +=1
                print(kontrol)
        try:
            #açıklama
            parametre1 = 0
            icerik=""
            while True:
                parametre1+=1
                icerik+=driver.find_element_by_xpath("//*[@id='job-details']/span/p["+str(parametre1)+"]").text
        except:
                try:
                    icerik+=driver.find_element_by_xpath("//*[@id='job-details']/span/ul[1]").text
                except:
                    pass
        try:
            parametre2=0
            while True:
                parametre2+=1
                icerik+=driver.find_element_by_xpath("//*[@id='job-details']/span/p["+str(parametre2)+"]").text
        except:
                try:
                    icerik+=driver.find_element_by_xpath("//*[@id='job-details']/span/ul[2]").text
                except:
                    pass
        if(icerik == ""):
            açıklama.append(None)
        else:
            açıklama.append(icerik)
        try:
            """print(element.text)
            print(ana)
            print(bas_say)
            print(cal_say)"""
            print(ksii.text)
        except:
            pass
    try:
        if(j>8 and j<sayfa_no-7):
            driver.find_element_by_xpath("(//*[@class='artdeco-pagination__indicator artdeco-pagination__indicator--number ember-view'])[6]").click()
            print("{}. sayfa".format(j))
            print("-----------------------------------------------------------")
        elif(j>=sayfa_no-7):
            driver.find_element_by_xpath("(//*[@class='artdeco-pagination__indicator artdeco-pagination__indicator--number ember-view'])["+str(j-(sayfa_no-10))+"]").click()
            print("{}. sayfa".format(j))
            print("-----------------------------------------------------------")
        else:
            driver.find_element_by_xpath("(//*[@class='artdeco-pagination__indicator artdeco-pagination__indicator--number ember-view'])["+str(j)+"]").click()
            print("{}. sayfa".format(j))
            print("-----------------------------------------------------------")
    except:
        pass

İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
İnternet
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Bankacılık
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Eğlence
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Lojistik ve Tedarik Zinciri
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Uzm

Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri Bilgisayar Yazılımı
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri Bilgisayar Yazılımı
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri Bilgisayar Yazılımı
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri Bilgisayar Yazılımı
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
3. sayfa
-----------------------------------------------------------
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri
İstihdam Türü
Tam Zamanlı
İş Alanları
Ürün Yönetimi Bilgi Teknolojisi Strateji/Planlama
Kıdem Düzeyi
Uzman
Sektör
Bilgi Teknolojisi ve Hizmetleri
İstihdam Türü
Tam Zamanlı
İş Alanları
Ürün Yönetimi Bilgi Teknolojisi Strateji/Planlama
K

İstihdam Türü
Geçici
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
8. sayfa
-----------------------------------------------------------
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Başlangıç Seviye
Sektör
Bilgi Teknolojisi ve Hizmetleri
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstih

İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Uzman
Sektör
Bankacılık
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
15. sayfa
-----------------------------------------------------------
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Sözleşmeli
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Diğer
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
16. sayfa
-----------------------------------------------------------
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Uzman
Sektör
Bilgi Te

İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
22. sayfa
-----------------------------------------------------------
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Başlangıç Seviye
Sektör
Bankacılık Bilgi Teknolojisi ve Hizmetleri
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Uzman
Sektör
Dinlence, Seyahat ve Turizm
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi Mühendislik Diğer
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Bilgisayar Yazılımı
İstihdam Türü
Tam Zamanlı
İş Alanları
İnsan Kaynakları
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Lojistik ve Tedarik Zinciri Bankacılık Bilgi Teknolojisi ve Hizmetleri
İstihdam Türü
Tam Zamanlı
İş Alanları
Analiz Uzmanı Mühendislik Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Uzman
Sektör
İnternet
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi

Kıdem Düzeyi
Başlangıç Seviye
Sektör
Bankacılık
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Bankacılık
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Uzman
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Sigorta
İstihdam Türü
Tam Zamanlı
İş Alanları
Genel İş
27. sayfa
-----------------------------------------------------------
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Üst Düzey Yönetici
Sektör
Nanoteknoloji
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendislik Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Başlangıç Seviye
Sektör
Bilgi Teknolojisi ve Hizmetleri Bilgisayar Yazılımı Hastane ve Sağlık
İstihdam Türü
Sözleşmeli
İş Alanları
Mühendislik Bilgi Teknolojisi
Kıdem Düzeyi
Başlangıç Seviye
Sektör
Savunma ve Uzay
İstihdam Türü
Tam Zamanlı
İş Alanları
Mühendi

Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
31. sayfa
-----------------------------------------------------------
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyon
İstihdam Türü
Tam Zamanlı
İş Alanları
Bilgi Teknolojisi
İstihdam Türü
Tam Zamanlı
Kıdem Düzeyi
Orta-Üst Düzey Yönetici
Sektör
Telekomünikasyo

Kıdem Düzeyi
Başlangıç Seviye
Sektör
Elektrik/Elektronik Üretimi Bilgisayar Yazılımı Tüketici Elektroniği
İstihdam Türü
Tam Zamanlı
İş Alanları
Müşteri Hizmetleri Bilgi Teknolojisi
Kıdem Düzeyi
Başlangıç Seviye
Sektör
Elektrik/Elektronik Üretimi Bilgisayar Yazılımı Tüketici Elektroniği
İstihdam Türü
Tam Zamanlı
İş Alanları
Müşteri Hizmetleri Bilgi Teknolojisi
34. sayfa
-----------------------------------------------------------
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


Şirket bilgileri ve konum bilgileri ana başlıkta bir arada bulunuyordu. Bunun için 2 adet boş listeye ihtiyacım var.

In [21]:
sirket = []
konum = []

Ana başık listesinin içinde var olan şirket ve konum bilgilerini ayırıyorum. Eğer scraping sonucunda elimde bir bilgi mevcut değilse None değer atıyorum.

In [22]:
for i in ana_baslik:
    try:
        sirket.append(i.split("\n")[1])
    except:
        sirket.append(None)
    try:
        konum.append(i.split("\n")[3])
    except:
        konum.append(None)

Bizim için kıdem düzeyi , istihdam türü , iş birimi ve sektör fazlasıyla önemliydi. Gerçekten elde edip etmediğimi kontrol ediyorum.

In [23]:
print(kıdemd_isb_sek_ist)

['İstihdam Türü\nTam Zamanlı', 'Kıdem Düzeyi\nUzman\nSektör\nBilgi Teknolojisi ve Hizmetleri\nİstihdam Türü\nTam Zamanlı\nİş Alanları\nMühendislik Bilgi Teknolojisi', 'Kıdem Düzeyi\nUzman\nSektör\nBilgi Teknolojisi ve Hizmetleri\nİstihdam Türü\nTam Zamanlı\nİş Alanları\nMühendislik Bilgi Teknolojisi', 'Kıdem Düzeyi\nUzman\nSektör\nİnternet\nİstihdam Türü\nTam Zamanlı\nİş Alanları\nMühendislik Bilgi Teknolojisi', 'Kıdem Düzeyi\nUzman\nSektör\nBankacılık\nİstihdam Türü\nTam Zamanlı\nİş Alanları\nBilgi Teknolojisi', 'Kıdem Düzeyi\nUzman\nSektör\nTelekomünikasyon\nİstihdam Türü\nTam Zamanlı\nİş Alanları\nBilgi Teknolojisi', 'Kıdem Düzeyi\nOrta-Üst Düzey Yönetici\nSektör\nEğlence\nİstihdam Türü\nTam Zamanlı\nİş Alanları\nMühendislik Bilgi Teknolojisi', 'İstihdam Türü\nTam Zamanlı', 'Kıdem Düzeyi\nOrta-Üst Düzey Yönetici\nSektör\nTelekomünikasyon\nİstihdam Türü\nTam Zamanlı\nİş Alanları\nBilgi Teknolojisi', 'Kıdem Düzeyi\nUzman\nSektör\nLojistik ve Tedarik Zinciri\nİstihdam Türü\nTam Zamanlı

Kıdem düzeyi , istihdam türü , iş birimi ve sektör bilgileri tek bir liste altındaydı. Bu bilgileri de ayrı ayrı elde etmem gerek.

In [24]:
kıdem_düzeyi = []
sektör = []
is_birimi = []
istihdam_türü = []

Aşağıda tasarladığım 4 adet kod bloğu ayrı ayrı bu bilgileri elde etmeme olanak sağladı. Yukarıda bu bilgileri elde edip etmediğimi kontrol ederken aynı zamanda ne şekilde elde ettiğimi de gördüm. Buna uygun bir tasarım gerçekleştirdim.

In [25]:
for i in kıdemd_isb_sek_ist:
    try:
        if(i.split("\n")[0].startswith("Kıdem")):
            kıdem_düzeyi.append(i.split("\n")[1])

        elif(i.split("\n")[2].startswith("Kıdem")):
            kıdem_düzeyi.append(i.split("\n")[3])

        elif(i.split("\n")[4].startswith("Kıdem")):
            kıdem_düzeyi.append(i.split("\n")[5])

        elif(i.split("\n")[6].startswith("Kıdem")):
            kıdem_düzeyi.append(i.split("\n")[7])
        else:
            kıdem_düzeyi.append(None)
    except:
        kıdem_düzeyi.append(None)

In [26]:
for i in kıdemd_isb_sek_ist:
    try:
        if(i.split("\n")[0].startswith("Sektör")):
            sektör.append(i.split("\n")[1])

        elif(i.split("\n")[2].startswith("Sektör")):
            sektör.append(i.split("\n")[3])

        elif(i.split("\n")[4].startswith("Sektör")):
            sektör.append(i.split("\n")[5])

        elif(i.split("\n")[6].startswith("Sektör")):
            sektör.append(i.split("\n")[7])
        else:
            sektör.append(None)
    except:
        sektör.append(None)

In [27]:
for i in kıdemd_isb_sek_ist:
    try:
        if(i.split("\n")[0].startswith("İstihdam")):
            istihdam_türü.append(i.split("\n")[1])

        elif(i.split("\n")[2].startswith("İstihdam")):
            istihdam_türü.append(i.split("\n")[3])

        elif(i.split("\n")[4].startswith("İstihdam")):
            istihdam_türü.append(i.split("\n")[5])

        elif(i.split("\n")[6].startswith("İstihdam")):
            istihdam_türü.append(i.split("\n")[7])
        else:
            istihdam_türü.append(None)
    except:
        istihdam_türü.append(None)

In [28]:
for i in kıdemd_isb_sek_ist:
    try:
        if(i.split("\n")[0].startswith("İş Alanları")):
            is_birimi.append(i.split("\n")[1])

        elif(i.split("\n")[2].startswith("İş Alanları")):
            is_birimi.append(i.split("\n")[3])

        elif(i.split("\n")[4].startswith("İş Alanları")):
            is_birimi.append(i.split("\n")[5])

        elif(i.split("\n")[6].startswith("İş Alanları")):
            is_birimi.append(i.split("\n")[7])
        else:
            is_birimi.append(None)
    except:
        is_birimi.append(None)

Kıdem düzeyini gerçekten ayırıp ayıramadığımı kontrol ediyorum.

In [29]:
print(kıdem_düzeyi)

[None, 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Orta-Üst Düzey Yönetici', None, 'Orta-Üst Düzey Yönetici', 'Uzman', None, 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', None, None, 'Uzman', 'Başlangıç Seviye', None, 'Uzman', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Orta-Üst Düzey Yönetici', 'Uzman', 'Uzman', 'Orta-Üst Düzey Yönetici', None, 'Uzman', None, 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzman', 'Uzm

Dataframe oluşturucağım için dataframe'in tüm sütunları birbirine eşit olmalı . Bunu kontrol ediyorum.

In [30]:
if((len(ana_baslik)==len(konum)) and (len(sirket)==len(pozisyon)) and (len(basvuru_sayisi) and len(calisan_sayisi)) and (len(açıklama)==len(istihdam_türü)) and (len(sektör)==len(is_birimi)) and (len(kıdem_düzeyi)==len(açıklama))):
    print(True)

True


Yukarıda kontrol ettim ama görsel olarak sayılarının eşit olduğunu da görmek istiyorum.

In [31]:
print(len(ana_baslik),len(konum),len(sirket),len(pozisyon),len(basvuru_sayisi),len(calisan_sayisi),len(açıklama),len(istihdam_türü),len(sektör),len(is_birimi),len(kıdem_düzeyi),len(açıklama))

875 875 875 875 875 875 875 875 875 875 875 875


Dataframe oluşturuyorum.

In [32]:
Yazılım = pd.DataFrame({'Şirket': sirket,
'Pozisyon':pozisyon,
'Başvuru Sayısı': basvuru_sayisi,
'Çalışan Sayısı': calisan_sayisi,
'Konum': konum,
'Açıklama':açıklama,
'İş Birimi':is_birimi,
'İstihdam Türü':istihdam_türü,
'Sektör':sektör,
'Pozisyon':pozisyon,
'Kıdem Düzeyi':kıdem_düzeyi
})

In [33]:
Yazılım

,Şirket,Pozisyon,Başvuru Sayısı,Çalışan Sayısı,Konum,Açıklama,İş Birimi,İstihdam Türü,Sektör,Kıdem Düzeyi
0,ROYEX YAZILIM MARKET YATIRIM DANIŞMANLIK TİC L...,Senior Software Engineer,8 başvuru,201-500 çalışan,"İstanbul, Türkiye","The ideal candidate is a self-motivated, multi...",None,Tam Zamanlı,None,None
1,NETAŞ,Yazılım Analiz ve Test Uzmanı,None,1.001-5.000 çalışan,Ankara,GENEL NİTELİKLERYetkinlikler:İŞ TANIMIÜniversi...,Mühendislik Bilgi Teknolojisi,Tam Zamanlı,Bilgi Teknolojisi ve Hizmetleri,Uzman
2,İnnova Bilişim,Test Uzmanı/İstanbul,43 başvuru,1.001-5.000 çalışan,"İstanbul, Türkiye",GENEL NİTELİKLER VE İŞ TANIMIİnnova Bilişim Çö...,Mühendislik Bilgi Teknolojisi,Tam Zamanlı,Bilgi Teknolojisi ve Hizmetleri,Uzman
3,Yemeksepeti,Test Mühendisi,94 başvuru,5.001-10.000 çalışan,"İstanbul, Türkiye",GENEL NİTELİKLERİŞ TANIMIDünyanın en büyük mut...,Mühendislik Bilgi Teknolojisi,Tam Zamanlı,İnternet,Uzman
4,VakıfBank,Yazılım Test Otomasyon Mühendisi,61 başvuru,10001+ çalışan,"İstanbul, Türkiye",Bankamız Bilgi Teknolojileri Uygulama Gelişti...,Bilgi Teknolojisi,Tam Zamanlı,Bankacılık,Uzman
...,...,...,...,...,...,...,...,...,...,...
870,None,Sayısal Tasarım Mühendisi (Tam Zamanlı),None,None,None,None,None,None,None,None
871,None,Sayısal Tasarım Mühendisi (Tam Zamanlı),None,None,None,None,None,None,None,None
872,None,Sayısal Tasarım Mühendisi (Tam Zamanlı),None,None,None,None,None,None,None,None
873,None,Sayısal Tasarım Mühendisi (Tam Zamanlı),None,None,None,None,None,None,None,None


Kaç adet birbirinin aynısı ilanım var bakıyorum.

In [34]:
Yazılım.duplicated().sum()

340

Tekrarlı ilanları çıkardığım zaman geriye kalan ilan sayısına bakıyorum. İlan sayıları da bizim için önemliydi.

In [35]:
Yazılım.drop_duplicates().shape

(535, 10)

Tekrar eden ilanları atıyorum

In [36]:
Yazılım.drop_duplicates(inplace=True)

Elimde kalan ilan sayısına tekrar bakıyorum.

In [37]:
Yazılım.shape

(535, 10)

In [38]:
Yazılım

,Şirket,Pozisyon,Başvuru Sayısı,Çalışan Sayısı,Konum,Açıklama,İş Birimi,İstihdam Türü,Sektör,Kıdem Düzeyi
0,ROYEX YAZILIM MARKET YATIRIM DANIŞMANLIK TİC L...,Senior Software Engineer,8 başvuru,201-500 çalışan,"İstanbul, Türkiye","The ideal candidate is a self-motivated, multi...",None,Tam Zamanlı,None,None
1,NETAŞ,Yazılım Analiz ve Test Uzmanı,None,1.001-5.000 çalışan,Ankara,GENEL NİTELİKLERYetkinlikler:İŞ TANIMIÜniversi...,Mühendislik Bilgi Teknolojisi,Tam Zamanlı,Bilgi Teknolojisi ve Hizmetleri,Uzman
2,İnnova Bilişim,Test Uzmanı/İstanbul,43 başvuru,1.001-5.000 çalışan,"İstanbul, Türkiye",GENEL NİTELİKLER VE İŞ TANIMIİnnova Bilişim Çö...,Mühendislik Bilgi Teknolojisi,Tam Zamanlı,Bilgi Teknolojisi ve Hizmetleri,Uzman
3,Yemeksepeti,Test Mühendisi,94 başvuru,5.001-10.000 çalışan,"İstanbul, Türkiye",GENEL NİTELİKLERİŞ TANIMIDünyanın en büyük mut...,Mühendislik Bilgi Teknolojisi,Tam Zamanlı,İnternet,Uzman
4,VakıfBank,Yazılım Test Otomasyon Mühendisi,61 başvuru,10001+ çalışan,"İstanbul, Türkiye",Bankamız Bilgi Teknolojileri Uygulama Gelişti...,Bilgi Teknolojisi,Tam Zamanlı,Bankacılık,Uzman
...,...,...,...,...,...,...,...,...,...,...
852,None,PHP Geliştirici,None,None,None,None,None,None,None,None
853,None,Proje Yöneticisi (Tam Zamanlı),None,None,None,None,None,None,None,None
854,None,Web Developer Php Yazılım Uzmanları,None,None,None,None,None,None,None,None
855,None,Büyük Veri Takım Lideri (Tam Zamanlı),None,None,None,None,None,None,None,None


Hangi sütunlarda kaç addet None değere sahibim buna bakıyorum.

In [39]:
Yazılım.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 535 entries, 0 to 856
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Şirket          516 non-null    object
 1   Pozisyon        531 non-null    object
 2   Başvuru Sayısı  512 non-null    object
 3   Çalışan Sayısı  479 non-null    object
 4   Konum           512 non-null    object
 5   Açıklama        466 non-null    object
 6   İş Birimi       219 non-null    object
 7   İstihdam Türü   515 non-null    object
 8   Sektör          219 non-null    object
 9   Kıdem Düzeyi    218 non-null    object
dtypes: object(10)
memory usage: 46.0+ KB


Son olarak elde ettiğim dataframe'i xls dosyası olarak kaydediyorum.

In [40]:
Yazılım.to_excel("Yazılım_Son_Nisan_23.xls",encoding="utf-16")